# BERT

In [ ]:
#TODO: usare LabelEncoder

In [1]:
pip install import_ipynb

Note: you may need to restart the kernel to use updated packages.


In [2]:
import re
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset, SubsetRandomSampler
from sklearn.model_selection import StratifiedKFold
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from tqdm import tqdm


In [3]:
# in alternativa usare %%writefile per creare un file py e importare questo
import sys
sys.path.append('..')

import import_ipynb
from data_preparation import Preprocessing

importing Jupyter notebook from /Users/alessia/UniProjects/HLT/src/classifiers/../data_preparation/Preprocessing.ipynb


In [4]:
df = pd.read_csv('../../data/updated_tweets.csv')

## Preprocessing

### Normalize

In [5]:
def normalize_tweet_BERT(tweet):
    tweet = Preprocessing.remove_links_mentions(tweet)
    tweet = tweet.lower()
    tweet = Preprocessing.remove_hashtag(tweet)
    tweet = Preprocessing.remove_special_characters(tweet)
 
    tweet = Preprocessing.remove_spaces(tweet)
    tweet = Preprocessing.remove_textual_emojis(tweet)
    tweet = Preprocessing.remove_not_ASCII(tweet)

    return tweet

In [6]:
#df.tweet_text = [normalize_tweet_BERT(tweet) for tweet in df.tweet_text]

df['tweet_text'] = df['tweet_text'].apply(normalize_tweet_BERT)

In [7]:
df = Preprocessing.clean_normalized_df(df)

In [8]:
def normalize_label(c_type):
    if c_type == "not_cyberbullying":
        c_type = 0
    elif c_type == "age":
        c_type=1
    elif c_type == "gender":
        c_type=2
    elif c_type == "ethnicity":
        c_type=3
    elif c_type == "religion":
        c_type=4
    return c_type

df['cyberbullying_type'] = df['cyberbullying_type'].apply(normalize_label)
df

,tweet_text,cyberbullying_type
0,"in other words katandandre, your food was crap...",0
1,why is aussietv so white?,0
2,a classy whore? or more red velvet cupcakes?,0
3,"meh. thanks for the heads up, but not too con...",0
4,this is an isis account pretending to be a kur...,0
...,...,...
39828,"black ppl aren't expected to do anything, depe...",3
39829,turner did not withhold his disappointment. tu...,3
39830,i swear to god. this dumb nigger bitch. i have...,3
39831,yea fuck you rt if youre a nigger fucking unfo...,3


### Dataset split

In [9]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenizzazione e codifica
input_ids = []
attention_masks = []

for tweet in df['tweet_text']:
    encoded_dict = tokenizer.encode_plus(
        tweet,
        add_special_tokens=True,
        truncation=True,
        max_length=128,
        padding='max_length',
        return_attention_mask=True,
        return_tensors='pt',
    )
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(df['cyberbullying_type'].values)
dataset = TensorDataset(input_ids, attention_masks, labels)

In [10]:
print(torch.cuda.is_available())
print(torch.backends.mps.is_available())

False
True


In [11]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device("mps") #su mac
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

for fold, (train_idx, val_idx) in enumerate(skf.split(input_ids, labels)):
    train_sampler = SubsetRandomSampler(train_idx)
    val_sampler = SubsetRandomSampler(val_idx)
    train_loader = DataLoader(dataset, batch_size=16, sampler=train_sampler)
    val_loader = DataLoader(dataset, batch_size=16, sampler=val_sampler)

    model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5)
    model.to(device)
    optimizer = AdamW(model.parameters(), lr=5e-5, weight_decay=0.01)

    for epoch in range(3):  # Numero di epoche
        model.train()
        total_loss = 0
        for batch in tqdm(train_loader, desc=f"Training Fold {fold+1}, Epoch {epoch+1}"):
            batch = {k: v.to(device) for k, v in zip(['input_ids', 'attention_mask', 'labels'], batch)}
            model.zero_grad()
            outputs = model(**batch)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        avg_loss = total_loss / len(train_loader)
        print(f"Average Loss: {avg_loss:.2f}")

        model.eval()
        total_accuracy = 0
        for batch in val_loader:
            batch = {k: v.to(device) for k, v in zip(['input_ids', 'attention_mask', 'labels'], batch)}
            with torch.no_grad():
                outputs = model(**batch)
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1).flatten()
            total_accuracy += (preds == batch['labels']).cpu().numpy().mean()

        print(f"Validation Accuracy: {total_accuracy / len(val_loader):.4f}")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/anaconda3/lib/python3.11/site-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Training Fold 1, Epoch 1: 100%|█████████████| 1959/1959 [44:58<00:00,  1.38s/it]


Average Loss: 0.24
Validation Accuracy: 0.9338


Training Fold 1, Epoch 2: 100%|█████████████| 1959/1959 [42:54<00:00,  1.31s/it]


Average Loss: 0.14
Validation Accuracy: 0.9437


Training Fold 1, Epoch 3: 100%|███████████| 1959/1959 [1:03:17<00:00,  1.94s/it]


...

Average Loss: 0.25
Validation Accuracy: 0.9356


Training Fold 5, Epoch 2: 100%|█████████████| 1960/1960 [36:35<00:00,  1.12s/it]


Average Loss: 0.15
Validation Accuracy: 0.9386


Training Fold 5, Epoch 3: 100%|█████████████| 1960/1960 [36:35<00:00,  1.12s/it]


Average Loss: 0.10
Validation Accuracy: 0.9358


In [12]:
# Salvataggio del modello addestrato
model_save_path = "bert_for_cyberbullying.pth"
torch.save(model.state_dict(), model_save_path)


-------------